# ***Libraries & Tools***

In [1]:
import fitz  # PyMuPDF
import requests
import io
import re
import os
import math
from tqdm import tqdm 
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor

# ***PDF Extraction & Analysis***

## ***Helper Functions & General Variables***

In [5]:
def download_pdf(paper_id):
    response = requests.get(f"https://arxiv.org/pdf/{paper_id}")
    if response.status_code == 200:
        return io.BytesIO(response.content)
    else:
        #raise Exception(f"Failed to download PDF. Status code: {response.status_code}")
        return None 

def extract_text_from_pdf(pdf):
    try:
        doc = fitz.open(pdf) # If we're downloading th PDFs use: fitz.open("pdf", pdf)
        text = ""
        for page in doc:
            text += page.get_text("text") + "\n"
        return text if text.strip() else None  # Return None if empty
    except Exception as e:
        #print(f"❌ Failed to extract text from {pdf}: {e}")
        return None  # Failed to extract text

def extract_introduction(text, word_limit=500):
    lines = text.split("\n")
    introduction_found = False
    extracted_text = []
    word_count = 0
    
    #section_pattern = re.compile(r'^\s*(\d+\.?|[IVXLCDM]+\.?|[A-Z]\.?)\s+(INTRODUCTION|Introduction|introduction)\s*$')
    section_pattern = re.compile(r'^\s*(?:\d+\.?|[IVXLCDM]+\.?|[A-Z]\.?)?\s*(INTRODUCTION|Introduction|introduction)\s*$')

    
    for line in lines:
        if not introduction_found:
            if section_pattern.match(line):
                introduction_found = True
        else:
            #words = line.split()
            words = re.findall(r"\b\w+\b", line)
            try:
                if word_count + len(words) > word_limit:
                    # Find the position of the last word to include
                    remaining_words = word_limit - word_count
                    last_word = words[remaining_words - 1]  # Last word to keep

                    # Find where this last word appears in the original line
                    last_word_index = line.find(last_word) + len(last_word)

                    # Append the original line up to this point (preserving punctuation)
                    extracted_text.append(line[:last_word_index])
                    break
                else:
                    extracted_text.append(line)
                    word_count += len(words)
            except Exception:
                break  
    
    cleaned_text = re.sub(r'(\w+)-\s+(\w+)', r'\1\2', " ".join(extracted_text))
    return cleaned_text.strip() if cleaned_text.strip() else None  # None if intro not found


def update_output_file(output_file, successful_retries):
    # Read the file into a list
    with open(output_file, "r") as f:
        lines = f.readlines()  # Read all lines

    # Update the correct indices
    for index, new_text in successful_retries:
        if 0 <= index < len(lines):  # Ensure index is valid
            lines[index] = new_text.strip() + "\n" 
    
    # Write back the updated lines
    with open(output_file, "w") as f:
        f.writelines(lines)


def concatenate_files(file1, file2, output_file):
    with open(file1, 'r', encoding='utf-8') as f1, open(file2, 'r', encoding='utf-8') as f2, open(output_file, 'w', encoding='utf-8') as out:
        for line1, line2 in zip(f1, f2):
            if "FAILED" in line2:
                out.write(f"{line1.strip()}\n")
            else:
                out.write(f"{line1.strip()}. {line2.strip()}\n")

In [4]:
id_file = "graph-v2/Node_IDs.txt"  # File containing one ID per line

output_file = "graph-v2/ANC_500.txt"

## ***Serial***

In [6]:
# Function to process PDFs stored in directories
def process_pdfs(pdf_dirs, results):
    successes = 0
    failures = []

    for directory in pdf_dirs:
        for filename in tqdm(os.listdir(directory), desc="Processing files", unit="file"):
            if filename.endswith(".pdf"):
                pdf_path = os.path.join(directory, filename)

                text = extract_text_from_pdf(pdf_path)
                if text is None:
                    #print(f"Skipping {filename}: Failed to extract text")
                    failures.append(filename.replace("_", "/").replace(".pdf", ""))
                    continue

                intro_text = extract_introduction(text)
                if intro_text is None:
                    #print(f"Skipping {filename}: No introduction found")
                    failures.append(filename.replace("_", "/").replace(".pdf", ""))
                    continue

                results[filename.replace("_", "/").replace(".pdf", "")] = intro_text
                successes += 1
                #print(f"✅ Extracted Introduction from {filename}")

    return results, successes, failures  # Dictionary { "filename.pdf": "Introduction text" }



# Function to process individual papers
def process_pdf(paper_id):
    try:
        pdf_stream = download_pdf(paper_id)
        if not pdf_stream:
            print(f"{paper_id} failed to download")
            return None
        
        text = extract_text_from_pdf(pdf_stream)
        if text is None: 
            print(f"Failed to extract text from {paper_id}")
            return None
        
        introduction_text = extract_introduction(text)
        if introduction_text is None:
            print(f"Failed to extract text from introduction section for {paper_id}")
            return None
        
        return introduction_text
    except Exception as e:
        return None #f"Error: {str(e)}"


In [ ]:
paper_ids = ["0704.1274", "0704.1028", "0704.0954", "0704.1308"] # For selected papers
successful_retries = []

for paper_id in paper_ids: # for idx, paper_id in failed_papers[0]:
    intro_text = process_pdf(paper_id)
    if intro_text:
        print(f"\n==================================================")
        print(intro_text)
        print(f"==================================================\n")
        #successful_retries.append((idx, intro_text)) # "successful_retries" will be used with "failed_papers" 

In [7]:
pdf_dirs = ['Papers 1', 'Papers 2', 'Papers 3', 'Papers 4', 'Papers 5', 'Papers 6']

results = {}

with open(id_file, "r") as f:
    paper_ids = {line.strip() for line in f}  # Use a set for faster lookups

for paper_id in paper_ids:
    results[paper_id] = "FAILED"

In [8]:
paper_ids

{'1809.01604',
 'cmp-lg/9406023',
 '1307.1630',
 'cs/0511028',
 '1507.05122',
 '1709.01305',
 '1811.01721',
 '1307.6458',
 '1807.07247',
 '1812.10924',
 '1502.06895',
 '1605.02401',
 '1206.5253',
 '1711.04731',
 '1811.08764',
 '1703.08580',
 '1806.03332',
 '1808.07733',
 '1605.06492',
 '1901.03775',
 '1812.02207',
 '1803.00384',
 '1405.5864',
 '1806.06237',
 '1812.07858',
 '1504.02141',
 '1603.05359',
 '1803.06077',
 '1803.08554',
 '1604.04879',
 '0711.1056',
 '1804.02508',
 '1802.05581',
 '1808.06645',
 '1402.2224',
 '1509.01951',
 '1510.08583',
 '1805.01220',
 '1501.06206',
 '1809.06065',
 '0911.4530',
 '1704.01523',
 '1809.08343',
 '1811.09678',
 '1002.2780',
 '1204.4249',
 '1610.00843',
 '1511.09180',
 '1808.04816',
 '1710.08167',
 '1511.06566',
 '1809.00970',
 '1808.07456',
 '1901.06086',
 '1309.5843',
 '1705.09407',
 '0801.3272',
 '1605.09519',
 '1303.4085',
 '1809.01733',
 '1809.10934',
 '1707.01825',
 '1603.02366',
 '1806.08698',
 '1504.00353',
 '1801.01627',
 '1804.07942',
 '1

In [9]:
results

{'1809.01604': 'FAILED',
 'cmp-lg/9406023': 'FAILED',
 '1307.1630': 'FAILED',
 'cs/0511028': 'FAILED',
 '1507.05122': 'FAILED',
 '1709.01305': 'FAILED',
 '1811.01721': 'FAILED',
 '1307.6458': 'FAILED',
 '1807.07247': 'FAILED',
 '1812.10924': 'FAILED',
 '1502.06895': 'FAILED',
 '1605.02401': 'FAILED',
 '1206.5253': 'FAILED',
 '1711.04731': 'FAILED',
 '1811.08764': 'FAILED',
 '1703.08580': 'FAILED',
 '1806.03332': 'FAILED',
 '1808.07733': 'FAILED',
 '1605.06492': 'FAILED',
 '1901.03775': 'FAILED',
 '1812.02207': 'FAILED',
 '1803.00384': 'FAILED',
 '1405.5864': 'FAILED',
 '1806.06237': 'FAILED',
 '1812.07858': 'FAILED',
 '1504.02141': 'FAILED',
 '1603.05359': 'FAILED',
 '1803.06077': 'FAILED',
 '1803.08554': 'FAILED',
 '1604.04879': 'FAILED',
 '0711.1056': 'FAILED',
 '1804.02508': 'FAILED',
 '1802.05581': 'FAILED',
 '1808.06645': 'FAILED',
 '1402.2224': 'FAILED',
 '1509.01951': 'FAILED',
 '1510.08583': 'FAILED',
 '1805.01220': 'FAILED',
 '1501.06206': 'FAILED',
 '1809.06065': 'FAILED',
 '

In [10]:
print(len(paper_ids))
print(len(results))

52596
52596


In [11]:
results, successes, failures = process_pdfs(pdf_dirs, results)

print(f'Successes: {successes}. Failures: {len(failures)}. Node IDs: {len(paper_ids)}.')

Processing files:   2%|▏         | 215/9557 [00:19<19:42,  7.90file/s]

MuPDF error: syntax error: could not parse color space (275 0 R)

MuPDF error: syntax error: could not parse color space (392 0 R)

MuPDF error: syntax error: could not parse color space (694 0 R)

MuPDF error: syntax error: could not parse color space (1005 0 R)

MuPDF error: syntax error: could not parse color space (1440 0 R)



Processing files:   6%|▌         | 587/9557 [00:49<08:54, 16.79file/s]

MuPDF error: syntax error: could not parse color space (427 0 R)

MuPDF error: syntax error: could not parse color space (660 0 R)

MuPDF error: syntax error: could not parse color space (834 0 R)

MuPDF error: syntax error: could not parse color space (855 0 R)

MuPDF error: syntax error: could not parse color space (875 0 R)



Processing files:   5%|▌         | 455/8952 [00:31<09:23, 15.08file/s]

...

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations

MuPDF error: unsupported error: cannot create appearance stream for Screen annotations



Processing files:  97%|█████████▋| 6917/7143 [08:46<00:12, 17.92file/s]

MuPDF error: syntax error: cannot find XObject resource 'arial-minus'

MuPDF error: syntax error: cannot find XObject resource 'arial-minus'



Processing files: 100%|██████████| 7143/7143 [09:02<00:00, 13.18file/s]

Successes: 50004. Failures: 2268. Node IDs: 52596.


Successes: 50004. Failures: 2268. Node IDs: 52596.


In [12]:
results

{'1809.01604': 'Merging datasets is a key operation for data analytics. A frequent requirement for merging is joining across columns that have different surface forms for the same entity. For instance, the name of a person might be represented as Douglas Adams, Douglas Noel Adams, D. Adams or Adams, Douglas. Similarly, ontology alignment can require recognizing distinct surface forms of the same entity, especially when ontologies are independently developed. This problem occurs for many entity types such as people’s names, company names, addresses, product descriptions, conference venues, or even people’s faces. Data management systems have however, largely focussed solely on equi-joins, where string or numeric equality determines which rows should be joined, because such joins are efﬁcient. We propose a different approach to joining different surface representations of the same entity, inspired by recent advances in deep learning. Our approach depends on (a) mapping surface forms into

In [13]:
with open(output_file, "w", encoding='utf-8') as f:
    for filename, text in results.items():
        f.write(text + "\n")

In [ ]:
print(failures)

In [14]:
concatenate_files('graph-v2/data-v2.txt', 'graph-v2/ANC_500.txt', 'graph-v2/data-v2-500.txt')

In [15]:
for txtf in ['data-v2-500.txt']:
    total_word_count = 0
    total_lines = 0

    with open(f'graph-v2/{txtf}', 'r', encoding='utf-8') as file:
        for line in file:
            total_word_count += len(line.split())
            total_lines += 1

    mean_word_count = total_word_count / total_lines if total_lines > 0 else 0
    print(f'=== {txtf} ===')
    print("Mean word count:", math.ceil(mean_word_count))
    print()

=== data-v2-500.txt ===
Mean word count: 606



## ***Parallel*** (Only works for downloading and processing papers)

In [ ]:
def process_range(paper_ids, start, end):
    results = [(i, "FAILED") for i in range(start, end)]
    failed_papers = [[] for _ in range(3)]  # Track failures: [download, extraction, introduction]
    
    #for i, paper_id in enumerate(paper_ids, start=start):
    for i, paper_id in enumerate(tqdm(paper_ids, total=len(paper_ids), desc="Processing Papers"), start=start):

        pdf_stream = download_pdf(paper_id)
        if not pdf_stream:
            failed_papers[0].append((i, paper_id))
            continue
        
        text = extract_text_from_pdf(pdf_stream)
        if text is None:
            failed_papers[1].append((i, paper_id))
            continue
        
        introduction_text = extract_introduction(text)
        if introduction_text is None:
            failed_papers[2].append((i, paper_id))
            continue
        
        results[i - start] = (i, introduction_text)
    
    return results, failed_papers

def process_papers_parallel(id_file, ranges, output_file, max_workers=5):
    with open(id_file, "r") as f:
        paper_ids = [line.strip() for line in f]
    
    failed_papers = [[] for _ in range(3)]  # Track failures: [download, extraction, introduction]
    results = ["FAILED"] * len(paper_ids)  # Initialize output list with "FAILED"
    
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        future_to_range = {executor.submit(process_range, paper_ids[r[0]:r[1]], r[0], r[1]): r for r in ranges}
        
        for future in future_to_range:
            range_results, range_failures = future.result()
            for i, text in range_results:
                results[i] = text
            for j in range(3):
                failed_papers[j].extend(range_failures[j])
    

    return failed_papers, results

In [ ]:
# Create ranges
num_ids = 52596
batch_size = int(num_ids / 6) # 6 is the number of processes  

ranges = []

start = 0
while start < num_ids:
    end = min(start + batch_size, num_ids)
    ranges.append([start, end])
    start = end

ranges[-2][1] = ranges[-1][1]
del ranges[-1]
print(ranges)

[[0, 5259], [5259, 10518], [10518, 15777], [15777, 21036], [21036, 26295], [26295, 31554], [31554, 36813], [36813, 42072], [42072, 47331], [47331, 52596]]


In [ ]:
failed_papers, results = process_papers_parallel(id_file, ranges, output_file)

In [ ]:
with open(output_file, "w", encoding='utf-8') as f:
    for text in results:
        f.write(text + "\n")
    
with open("graph-v2/Failed_papers.txt", "w") as f:
    f.write("Failed Downloads:\n")
    for idx, pid in failed_papers[0]:
        f.write(f"{idx}: {pid}\n")
    f.write("\nFailed Extraction:\n")
    for idx, pid in failed_papers[1]:
        f.write(f"{idx}: {pid}\n")
    f.write("\nFailed Introduction Detection:\n")
    for idx, pid in failed_papers[2]:
        f.write(f"{idx}: {pid}\n")